In [26]:
import os

In [ ]:
%pwd

In [ ]:
# os.chdir("../")
%pwd

In [29]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_col: str

In [30]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [35]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH            
    ):
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        self.schema= read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_training
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config= ModelTrainerConfig(
            root_dir= config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name= config.model_name,
            alpha= params.alpha,
            l1_ratio= params.l1_ratio, 
            target_col= schema.name
        )
        return model_trainer_config


In [36]:
from src.datascience import logger

import os 

import pandas as pd
from sklearn.linear_model import ElasticNet
import joblib
from sklearn.metrics import accuracy_score

In [37]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data= pd.read_csv(self.config.train_data_path)
        test_data= pd.read_csv(self.config.test_data_path)

        train_X= train_data.drop([self.config.target_col], axis=1)
        test_X= test_data.drop([self.config.target_col], axis=1)

        train_y= train_data[[self.config.target_col]]
        test_y= test_data[[self.config.target_col]]

        lr= ElasticNet(
            alpha= self.config.alpha,
            l1_ratio= self.config.l1_ratio,
            random_state= 42
        )
        lr.fit(train_X, train_y)
        
        # y_predict= lr.predict(test_X)
        # accuracy= accuracy_score(test_y, y_predict)

        logger.info("Model Trained Successfully.")
        # logger.info(f"Model accuracy: {accuracy}")

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))
        logger.info(f"Model is dumped at {os.path.join(self.config.root_dir, self.config.model_name)}")

In [38]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config= model_trainer_config)
    model_trainer.train()

except Exception as e:
    raise e

[2024-12-30 22:55:38,094: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-30 22:55:38,094: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-30 22:55:38,103: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-30 22:55:38,103: INFO: common: created directory at: artifacts]
[2024-12-30 22:55:38,103: INFO: common: created directory at: artifacts/model_training]
[2024-12-30 22:55:38,121: INFO: 3714723069: Model Trained Successfully.]
[2024-12-30 22:55:38,127: INFO: 3714723069: Model is dumped at artifacts/model_training\model.joblib]
